# TCP Client

##  Exemplu 1: clientul se conecteaza la server si primeste un mesaj

In [1]:
import socket

# Adresa IP Server - localhost (daca rulam pe acelasi PC)
HOST = "127.0.0.1"
# Port server
PORT = 34512  

# Dimensiunea maxima care va fi primita pe socket de la client
BUFFER_SIZE  = 1024

# Deschiderea unui socket TCP
with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
    # Conectare la server
    tcp_sock.connect( (HOST, PORT) )
    # Blocare executie si asteptare mesaj de la server; Se vor citi cel mult 1024 de octeti
    data = tcp_sock.recv( BUFFER_SIZE )
    # Decodare octeti primiti in string
    decoded_data  = data.decode()
    print( "Primit de la server: ", decoded_data )
    
    # Inchidere socket
    tcp_sock.close()

Primit de la server:  Salutari, client


##  Exemplu 2: clientul 1 se conecteaza la server, trimite mesaje si primeste raspuns

In [4]:
import socket
import time

HOST = "127.0.0.1"
PORT = 34512

BUFFER_SIZE  = 1024

for i in range(0, 100):
    with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
        tcp_sock.connect( (HOST, PORT) )
        
        # Construire mesaj catre server
        message = "Sunt clientul 1"
        # Codificare mesaj ca UTF-8
        message_bytes = str.encode( message )
        # Trimiterea intregului mesaj pe socket
        tcp_sock.sendall( message_bytes )
        # Asteptare raspuns de la server
        data = tcp_sock.recv( BUFFER_SIZE )
        decoded_data  = data.decode()
    
        print( "Primit de la server: ", decoded_data )
    tcp_sock.close()
    #time.sleep(1)

Primit de la server:  Salut client 1, esti conexiunea numarul 9
Primit de la server:  Salut client 1, esti conexiunea numarul 10
Primit de la server:  Salut client 1, esti conexiunea numarul 11
Primit de la server:  Salut client 1, esti conexiunea numarul 12


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

##  De rezolvat:
#### Scrieti si rulati codul de mai sus eliminand delay-ul de o secunda ( *time.sleep(1)* ), atat pentru clientul 1, cat si pentru clientul 2
#### Pentru server, folositi acelasi cod din Exemplul 2
#### Capturati in Wireshark traficul si observati ce se intampla

## Exemplu 3.1: clientul citeste o imagine dintr-un fisier si o trimite la server

In [15]:
import socket

HOST = "127.0.0.1"
PORT = 34512

# Dimensiunea imaginii
BUFFER_SIZE  = 2984

file = open('cat.png', 'rb')
# Citirea imaginii din fisier
image_file = file.read( BUFFER_SIZE )

with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
    tcp_sock.connect( (HOST, PORT) )
    
    # Trimiterea imaginii pe socket
    tcp_sock.send( image_file )

    file.close()
    tcp_sock.close()

## Exemplu 3.2: clientul citeste o imagine de la camera si o trimite la server

In [2]:
import socket
import cv2

HOST = "127.0.0.1"
PORT = 34512

# Dimensiunea imaginii redimensionate: 120*160
BUFFER_SIZE  = 19200

cap = cv2.VideoCapture(0)
stop = 'a'

with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
    tcp_sock.connect( (HOST, PORT) )
    
    # Citire frame de la camera
    ret, frame = cap.read()
    # Redimensionare frame
    frame = cv2.resize( frame, (160, 120) )
    # Codare imagine citita ca jpg, in bytes
    img_str = cv2.imencode('.jpg', frame)[1].tobytes()
    
    # Trimitere imagine
    tcp_sock.send( img_str )
    
    # Asteptare raspuns de la server, decodare raspuns si afisare
    data = tcp_sock.recv( 1024 )
    decoded_data = data.decode()
    
 
    if( decoded_data == 'primit' ):
        print('Imaginea a fost trimisa cu succes')
    if( decoded_data == 'stop' ):
        print('STOP')
    
    tcp_sock.close()
    
cap.release()

STOP


## De rezolvat: 
#### Cititi imagini de la camera si trimiteti catre client, pana cand primiti ca raspuns 'stop'

# UDP Client

## Exemplu 4: clientul trimite un mesaj la server si asteapta un raspuns

In [ ]:
import socket

HOST = "127.0.0.1"
PORT = 34512

BUFFER_SIZE = 1024

# Crearea unui socket UDP la client
with socket.socket( socket.AF_INET, type=socket.SOCK_DGRAM ) as udp_sock:
    # Construire mesaj de trimis
    message = "Salutari de la Clientul UDP 1"
    message_encoded = str.encode(message)

    # Trimitere mesaj catre server
    udp_sock.sendto( message_encoded, (HOST, PORT) )
    
    # Ascultare pentru datagrame incoming, de dimensiune maxima BUFFER_SIZE
    server_message = udp_sock.recvfrom( BUFFER_SIZE )
    server_message_decoded = server_message[0].decode()

    print("Mesaj de la server: ", server_message_decoded)
    
    udp_sock.close()

## De rezolvat: 
#### Pe baza codului de mai sus, scrieti un client UDP care sa deschida un socket catre server de 10 ori, sa trimita un mesaj de fiecare data si sa astepte un raspuns, la un interval de jumatate de secunda